In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics

In [ ]:
df = pd.read_csv('diabetes2.csv')

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
# df = df.dropna()
# df.info()

df['Glucose'].fillna(df['Glucose'].mean(), inplace = True)
df['BloodPressure'].fillna(df['BloodPressure'].mean(), inplace = True)
df['SkinThickness'].fillna(df['SkinThickness'].mean(), inplace = True)
df['Insulin'].fillna(df['Insulin'].mean(), inplace = True)
df['BMI'].fillna(df['BMI'].mean(), inplace = True)

In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31,0
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.00000,180.000000,32.9,0.171,63,0
764,2,122.0,70.0,27.00000,155.548223,36.8,0.340,27,0
765,5,121.0,72.0,23.00000,112.000000,26.2,0.245,30,0
766,1,126.0,60.0,29.15342,155.548223,30.1,0.349,47,1


In [ ]:
columns_to_scale = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
numeric_data = df[columns_to_scale]
scaler = StandardScaler()
scaler.fit(numeric_data)
scaled_data = scaler.transform(numeric_data)
df[columns_to_scale] = scaled_data

In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.639947,0.865108,-0.033518,6.655021e-01,-3.345079e-16,0.166292,0.468492,1.425995,1
1,-0.844885,-1.206162,-0.529859,-1.746338e-02,-3.345079e-16,-0.852531,-0.365061,-0.190672,0
2,1.233880,2.015813,-0.695306,8.087936e-16,-3.345079e-16,-1.332833,0.604397,-0.105584,1
3,-0.844885,-1.074652,-0.529859,-7.004289e-01,-7.243887e-01,-0.634212,-0.920763,-1.041549,0
4,-1.141852,0.503458,-2.680669,6.655021e-01,1.465506e-01,1.548980,5.484909,-0.020496,1
...,...,...,...,...,...,...,...,...,...
763,1.827813,-0.680125,0.297376,2.145261e+00,2.877840e-01,0.064409,-0.908682,2.532136,0
764,-0.547919,0.010298,-0.198965,-2.451185e-01,-3.345079e-16,0.632039,-0.398282,-0.531023,0
765,0.342981,-0.022579,-0.033518,-7.004289e-01,-5.125386e-01,-0.910750,-0.685193,-0.275760,0
766,-0.844885,0.141808,-1.026200,8.087936e-16,-3.345079e-16,-0.343120,-0.371101,1.170732,1


In [ ]:
outcome_correlation = (df.corr()['Outcome']).sort_values()

In [ ]:
outcome_correlation

BloodPressure               0.166074
DiabetesPedigreeFunction    0.173844
Insulin                     0.214411
SkinThickness               0.215299
Pregnancies                 0.221898
Age                         0.238356
BMI                         0.311924
Glucose                     0.492928
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
df.drop(['BloodPressure','DiabetesPedigreeFunction'], inplace = True, axis = 1)

In [ ]:
df

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,Age,Outcome
0,0.639947,0.865108,6.655021e-01,-3.345079e-16,0.166292,1.425995,1
1,-0.844885,-1.206162,-1.746338e-02,-3.345079e-16,-0.852531,-0.190672,0
2,1.233880,2.015813,8.087936e-16,-3.345079e-16,-1.332833,-0.105584,1
3,-0.844885,-1.074652,-7.004289e-01,-7.243887e-01,-0.634212,-1.041549,0
4,-1.141852,0.503458,6.655021e-01,1.465506e-01,1.548980,-0.020496,1
...,...,...,...,...,...,...,...
763,1.827813,-0.680125,2.145261e+00,2.877840e-01,0.064409,2.532136,0
764,-0.547919,0.010298,-2.451185e-01,-3.345079e-16,0.632039,-0.531023,0
765,0.342981,-0.022579,-7.004289e-01,-5.125386e-01,-0.910750,-0.275760,0
766,-0.844885,0.141808,8.087936e-16,-3.345079e-16,-0.343120,1.170732,1


In [ ]:
df = df.sample(frac=1, random_state=50)
y = df['Outcome']
X = df.drop('Outcome', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 50)

print(X_train.shape)
print(y_train.shape)

(614, 6)
(614,)


In [ ]:
model_lg = LogisticRegression()

model_lg.fit(X_train, y_train)
y_pred = model_lg.predict(X_test)
model_lg_score = model_lg.score(X_test, y_test)

print(model_lg_score)

0.7987012987012987


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7987012987012987
Precision: 0.7608695652173914
Recall: 0.6363636363636364
F1 Score: 0.693069306930693


In [ ]:
param_grid_lg = {'solver': ['saga', 'liblinear'],
                  'penalty': ['l1','l2'],
                 'C': [0.001,0.01,0.1,1,10,100,1000],
                  'max_iter' : [10,100,500]}

lg_grid = GridSearchCV(estimator=LogisticRegression(),  # the model
                            param_grid=param_grid_lg,  # hyperparameter space
                             scoring='roc_auc',
                             cv = 5,
                             n_jobs = -1)  # default: 5 folder cross-validation

In [ ]:
lg_grid.fit(X_train, y_train)
print(lg_grid.best_params_)
best_model = lg_grid.best_estimator_

{'C': 0.1, 'max_iter': 10, 'penalty': 'l1', 'solver': 'saga'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8051948051948052
Precision: 0.7906976744186046
Recall: 0.6181818181818182
F1 Score: 0.693877551020408


In [ ]:
classifiers = {
    # 'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(random_state=50),
    'Logistic Regression': LogisticRegression()
}

# Define hyperparameters grid for each classifier
param_grids = {
    # 'SVM': {
    #     'C': [0.1, 1, 10, 100, 1000],
    #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale', 'auto'],
    #     'kernel': ['rbf','linear']
    # },
    'KNN': {
        'n_neighbors': [3,4,5,6,7,8,9,10],
        'weights': ['uniform', 'distance']
    },
    'Random Forest': {
      'n_estimators': [50, 100, 150],
      'max_depth': [None, 10, 20],
      'min_samples_split': [2, 5, 10],
      'max_features': ['sqrt', 'log2']
    },
    'HistGradientBoosting': {
        'learning_rate': [0.1, 0.01, 0.001],
        'max_iter': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    },
    'Logistic Regression':{'solver': ['saga', 'liblinear'],
                  'penalty': ['l1','l2'],
                 'C': [0.001,0.01,0.1,1,10,100,1000],
                  'max_iter' : [10,100,500]}
}

In [ ]:
results = {}
for name, classifier in classifiers.items():
    print("Training: ", name)
    gridSearch = GridSearchCV(estimator=classifier, param_grid=param_grids[name], cv=5, n_jobs=-1)
    gridSearch.fit(X_train, y_train)
    results[name] = gridSearch.best_estimator_

print(results)

Training:  KNN
Training:  Random Forest
Training:  Logistic Regression
{'KNN': KNeighborsClassifier(n_neighbors=7), 'Random Forest': RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=150,
                       random_state=50), 'Logistic Regression': LogisticRegression(C=1, max_iter=10, penalty='l1', solver='liblinear')}


In [ ]:
results

{'KNN': KNeighborsClassifier(n_neighbors=7),
 'Random Forest': RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=150,
                        random_state=50),
 'Logistic Regression': LogisticRegression(C=1, max_iter=10, penalty='l1', solver='liblinear')}

In [ ]:
for name, clf in results.items():
    print("\nResults for", name)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)


Results for KNN
Accuracy: 0.7727272727272727

Results for Random Forest
Accuracy: 0.7532467532467533

Results for Logistic Regression
Accuracy: 0.7987012987012987
